##### Cargue de librerias

In [1]:
from pycaret.clustering import *
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import plotly.express as px
import plotly.figure_factory as ff
import plotly.subplots as sp
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")

##### Conexion al DWH de Dispapeles y carga de bds adicionales

In [2]:
def df_clustering(consulta_SQL):
    ## Conexion al DWH
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()
    
    #Cargue de la data desde el ERP de Dispapeles y se guarda en df
    cursor.execute(consulta_SQL)
    rows = cursor.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

    return df

##### Funcion EDA

In [3]:
# def histograma(bd):
#     fig, ax = plt.subplots(1, 3, figsize= (15,5))
#     sns.hist(bd["Escala R"], ax= ax[0], color= "blue", fill= True)
#     sns.hist(bd["Escala F"], ax= ax[1], color= "red", fill= True)
#     sns.hist(bd["Escala M"], ax= ax[2], color= "yellow", fill= True)
#     fig.suptitle("Distribuciones del modelo RFM")
#     plt.show()

def histograma(bd):
    hist_data = [bd["Escala R"], bd["Escala F"], bd["Escala M"]]
    group_labels = ["Escala R", "Escala F", "Escala M"]
    fig = sp.make_subplots(rows=1, cols=3)
    
    for i in range(3):
        fig.add_trace(go.Histogram(x=hist_data[i], showlegend=False, histfunc="avg"), row=1, col=i+1)
    fig.update_layout(
        title="Histogramas de Escala R, Escala F y Escala M",
        xaxis=dict(title="Valor"),
        yaxis=dict(title="Frecuencia"),
        showlegend=False
    )
    fig.show()

# def scatter(bd):
#     fig, ax = plt.subplots(1, 1, figsize= (8,5))
#     sns.scatterplot(x= "Escala R", size= "Escala F", y= "Escala M", data= bd)
#     fig.suptitle("Scatter del modelo RFM")
#     plt.show()

def scatter(bd):
    fig = px.scatter(bd, x= "Escala R", y= "Escala F", color= "Escala M")
    fig.show()

# def matrix(bd, col):
#     corr_matrix = bd[col].corr()
#     fig, ax = plt.subplots(figsize=(8, 6))
#     sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, ax=ax)
#     plt.title("Matriz de correlación")
#     plt.show()

def matrix(bd, col):
    corr_matrix = bd[col].corr()
    fig = px.imshow(corr_matrix, text_auto=True)
    fig.show()
    
def EDA(bd):
    bd = bd.reset_index()
    print(f"Esta base de datos tiene {len(bd)} registros,")
    print(f"hace parte del distrito {bd['Codigo distrito'][0]}")
    print(f"y el tipo de zona es {bd['Nombre tipo zona'][0]}")
    
    print(" ")
    print("Describe de la base de datos")
    describe = bd.describe().applymap("{:,.0f}".format)
    print(describe[["Escala R", "Escala F", "Escala M"]][1:])
    
    print(" ")
    
    histograma(bd)
    scatter(bd)
    matrix(bd, ["Escala R", "Escala F", "Escala M"])

##### Funciones estandarizacion

In [4]:
# Crear una función para asignar etiquetas basadas en percentiles
def asignar_escala_R(valor, escalas_R= [24, 21, 15, 8, 0]):
    if valor >= escalas_R[0]:
        return 5
    elif valor >= escalas_R[1]:
        return 4
    elif valor >= escalas_R[2]:
        return 3
    elif valor >= escalas_R[3]:
        return 2
    else:
        return 1
    
def asignar_escala_F(valor, escalas_F= [24, 21, 15, 8, 0]):
    if valor >= escalas_F[0]:
        return 5
    elif valor >= escalas_F[1]:
        return 4
    elif valor >= escalas_F[2]:
        return 3
    elif valor >= escalas_F[3]:
        return 2
    else:
        return 1
    
def asignar_escala_M(valor):
    if valor <= escalas_M[1]:
        return 1
    elif valor <= escalas_M[2]:
        return 2
    elif valor <= escalas_M[3]:
        return 3
    elif valor <= escalas_M[4]:
        return 4
    else:
        return 5
    

##### Carga de la base de datos

In [5]:
# Consulta SQL para descargar la informacion a usar
codigo_SQL_vta = "SELECT A.DSTCOD AS 'Codigo distrito', B.VNDZNA AS 'Codigo zona actual', CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC) AS 'Nit cliente-sucursal', A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', (((2021 - MAX(A.VTAANO)) * 12 ) + (0 - MAX(A.VTAMES))) * -1 AS 'Recency', SUM(A.VTAVLRVTA) AS 'Monetary', COUNT( DISTINCT A.VTAMES) AS 'Frequency' FROM V_VTA_VTAHEC A LEFT JOIN	DIM_CLIENTES B ON A.PRMCOD = B.PRMCOD AND A.DSTCOD = B.DSTCOD AND A.VTANIT = B.CLTNIT AND A.VTASUC = B.CLTSUC WHERE A.PRMCOD = 1 AND A.DSTCOD NOT IN (30,35) AND A.VTAZNA < 96 AND A.VTAFCH BETWEEN '2021-01-01' AND '2022-12-31' GROUP BY A.DSTCOD, B.VNDZNA, CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC), A.VTAANO, A.VTAMES HAVING SUM(A.VTAVLRVTA) > 0"
codigo_SQL_rdo = "SELECT Distrito AS 'Codigo distrito', Zona AS 'Codigo zona actual', CONCAT(CONCAT(Cliente,'-'), Suc) AS 'Nit cliente-sucursal', YEAR(Fch_PagoRecaudo) AS 'Ano', MONTH(Fch_PagoRecaudo) AS 'Mes', (((2021 - MAX(YEAR(Fch_PagoRecaudo))) * 12 ) + (0 - MAX(MONTH(Fch_PagoRecaudo)))) * -1 AS 'Recency', SUM(SinVenc + [Venc.30] + [Venc.60] + [Venc.90] + [Venc.120] + [Venc++120]) 'Monetary', COUNT(DISTINCT MONTH(Fch_PagoRecaudo)) AS 'Frequency' FROM RECAUDOS WHERE Distrito NOT IN (30, 35) AND Fch_PagoRecaudo BETWEEN '2021-01-01' AND '2022-12-31' AND Empresa = 1 GROUP BY Distrito, Zona, CONCAT(CONCAT(Cliente,'-'), Suc), YEAR(Fch_PagoRecaudo), MONTH(Fch_PagoRecaudo) HAVING	SUM(SinVenc + [Venc.30] + [Venc.60] + [Venc.90] + [Venc.120] + [Venc++120]) > 0"

# Consulta adicional para establecer el tipo de zona
maestra_zonas = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/Análisis clustering/Maestra_zonas.csv', delimiter= ";")

In [22]:
df_vta = df_clustering(codigo_SQL_vta)
df_rdo = df_clustering(codigo_SQL_rdo)
# df = df_clustering(codigo_SQL)

In [21]:
df_vta

,Codigo distrito,Codigo zona actual,Nit cliente-sucursal,Ano,Mes,Recency,Monetary,Frequency,Nombre tipo zona
0,10,1,1077871537-1,2022,8,20,22599528,1,Artes graficas
1,10,1,17172243-1,2022,3,15,976752,1,Artes graficas
2,10,1,19322486-1,2021,2,2,1848088,1,Artes graficas
3,10,1,19322486-1,2022,1,13,4350769,1,Artes graficas
4,10,1,19322486-1,2022,8,20,1470204,1,Artes graficas
...,...,...,...,...,...,...,...,...,...
125935,90,68,93387740-1,2021,10,10,28515,1,Institucional
125936,90,68,93387740-1,2022,3,15,24943,1,Institucional
125937,90,68,93387740-1,2022,6,18,191452,1,Institucional
125938,90,68,93387740-1,2022,9,21,214812,1,Institucional


In [18]:
df_rdo["Monetary"].min()

Decimal('0.13')

##### Tranformaciones

In [20]:
# Conversion de tipo de la columna Monetary
df_vta["Monetary"] = df_vta["Monetary"].astype(int)
# df_rdo["Monetary"] = df_rdo["Monetary"].astype(int)

# Join con la maestra de zonas para determinar el tipo de zona
df_vta2 = pd.merge(df_vta, maestra_zonas, on= ["Codigo distrito", "Codigo zona actual"])
df_rdo2 = pd.merge(df_rdo, maestra_zonas, on= ["Codigo distrito", "Codigo zona actual"])

# Filtro de la informacion para excluir zonas no relevantes
list_filter = ['Artes graficas', 'TIG', 'V Horeca', 'Institucional', 'V SyE',
                'V Industria', 'Mayorista']
df_vta = df_vta2[df_vta2["Nombre tipo zona"].isin(list_filter)]
df_rdo = df_rdo2[df_rdo2["Nombre tipo zona"].isin(list_filter)]

# DF final agregada por tipo de zona
df_vta3 = df_vta.groupby(["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"]).agg({"Recency": np.max,
                                                                            "Monetary": np.sum,
                                                                            "Frequency": np.sum
                                                                            })
df_rdo3 = df_rdo.groupby(["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"]).agg({"Recency": np.max,
                                                                            "Monetary": np.sum,
                                                                            "Frequency": np.sum
                                                                            })
df_vta3 = df_vta3.reset_index(col_level= 1)
df_rdo3 = df_rdo3.reset_index(col_level= 1)

# df_3 = pd.merge(df_vta3, df_rdo3, on=["Codigo distrito", "Codigo zona actual", "nit"], how="left")

KeyError: 'Nombre tipo zona'

In [ ]:
# Se añade una key de Distrito-Nombretipozona para hacer mas facil la identificacion cuando se usa todo el pais
df_3['Distrito-Nombretipozona'] = df_3['Codigo distrito'].astype(str) + '-' + df_3['Nombre tipo zona']

# Aplicar la función para asignar etiquetas a una nueva columna "Etiqueta"
escalas_M = np.percentile(df_3["Monetary"].values, [0, 20, 50, 80, 95, 100])

df_est = df_3
df_est["Escala R"] = df_3["Recency"].apply(asignar_escala_R, escalas_R= [24, 21, 15, 8, 0])
df_est["Escala F"] = df_3["Frequency"].apply(asignar_escala_F, escalas_F= [24, 21, 15, 8, 0])
df_est["Escala M"] = df_3["Monetary"].apply(asignar_escala_M)
df_est = df_est.drop(["Recency", "Frequency", "Monetary"], axis=1)
df_est

##### Creacion, ajuste y descarga de los modelos de clustering por cada distrito-tipo de zona

In [ ]:
list_dtos = df_est["Distrito-Nombretipozona"].unique()
numeric_features = ["Escala R", "Escala F", "Escala M"]
ignore_columns = ["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal", "Distrito-Nombretipozona"]

#### EDA

In [ ]:
df_EDA = df_est[df_est["Distrito-Nombretipozona"] == '10-Artes graficas']

In [ ]:
EDA(df_EDA)

In [ ]:
# list_dtos = ['10-Artes graficas', '10-Institucional', '10-Mayorista', '10-TIG', '10-V Horeca', '10-V Industria', '10-V SyE']
# list_dtos = ['10-Artes graficas']

##### Un modelo

In [ ]:
# Variables iniciales
metrics = []
bd_predicted_all = pd.DataFrame()

# Fecha para guardar resultados
fecha = '12-04-23'

# Ciclo para realizar el modelo de clustering por cada tipo de zona del pais
for n, dto in enumerate(list_dtos):
    print(f"Procesando modelo {n+1} de {len(list_dtos)}. {(n+1) / (len(list_dtos)):.1%}")
    
    #BD filtrada
    df_dto = df_est[df_est["Distrito-Nombretipozona"] == dto]

    #Setup del modelo 
    s = setup(
                df_dto,
                verbose= False,
                preprocess= False,
                profile= False,
                ignore_features= ignore_columns,
                numeric_features= numeric_features
                )
    
    #Creacion y prediccion de modelo
    model_kmeans = create_model(
                                'kmeans',
                                num_clusters= 5,
                                verbose= False,
                                )
    pull()
    df_pred_km = predict_model(
                                model_kmeans,
                                data = df_dto
                                )
    
    # Reasignacion de los nombres de los clusters
    trans = df_pred_km.groupby(
                                ["Cluster"]).agg({
                                                "Escala R": np.mean, "Escala F": np.mean, "Escala M": np.mean,
                                            }).reset_index()
    trans["avg"] = (trans["Escala R"] + trans["Escala F"] + trans["Escala M"]) / 3
    trans = trans.sort_values(by= ["avg"], ascending= False)
    trans["Letra cluster"] = ["A", "B", "C", "D", "E"]
    trans = trans[["Cluster", "Letra cluster"]]
    
    df_pred_km = pd.merge(df_pred_km, trans, on= ["Cluster"])
    df_pred_km = df_pred_km.drop_duplicates(subset=["Escala R", "Escala F", "Escala M"])
    
    # Descarga del modelo
    bd_predicted = pd.merge(df_dto, df_pred_km, on= ["Escala R", "Escala F", "Escala M"])

    bd_predicted_all = pd.concat([bd_predicted_all, bd_predicted], ignore_index=  True)

    #Append a la variable incial de metricas
    metrics.append({
                    "Distrito-zona": dto,
                    "Silhouette": pull().Silhouette[0],
                    "Calinski-Harabasz": pull()["Calinski-Harabasz"][0],
                    "Davies-Bouldin": pull()["Davies-Bouldin"][0]
                    })
    # save_model(model_kmeans, f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Modelos {fecha}/Modelo {dto} {fecha}")

# Transformacion de la variable de metricas
metrics = pd.DataFrame(metrics)
metrics["Tipo zona"] = metrics["Distrito-zona"].str[3:]

print("Proceso completo 100%")

In [ ]:
# Descarga de bd predicha y metricas calculadas
bd_predicted_all.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",", sep= ";")
metrics.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Metricas {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",",  sep= ";")

In [ ]:
fig = px.scatter_3d(df_pred_km.sort_values(by= "Letra cluster"), x= "Escala R", y= "Escala F", z= "Escala M",
                    color= "Letra cluster")
fig.show()

In [ ]:
plot_model(model_kmeans, plot = 'cluster')

In [ ]:
plot_model(model_kmeans, plot = 'silhouette')

In [ ]:
plot_model(model_kmeans, plot = 'distribution')

In [ ]:
trans